### Reading the data

In this notebook we will inspect the digitized data, both data and MC, and also the geometry.
Lets start off with a data file. We will be using ROOT. 

The path+filename is: /eos/experiment/sndlhc/convertedData/physics/2022/run_004705/sndsw_raw-0010.root 

In [ ]:
import ROOT

In [ ]:
file = ROOT.TFile("/eos/experiment/sndlhc/convertedData/physics/2022/run_004705/sndsw_raw-0010.root")
tree = file.rawConv
tree.GetEntries()

To print the branch content of the TTree:

In [ ]:
tree.Print()

In [ ]:
tree.GetEvent(0) # get the first event in the file - this command reads the index in the file
tree.EventHeader.GetEventNumber()

Remember we opened partition number 10, so the 1st event there is actually the 10th M event for the run - the event counter doesn't reset with partitions! 
The index in the ROOT file however, does reset.

In [ ]:
tree.EventHeader.GetFillNumber()

The fill number can be used to check out fill infomration from the LHC supertables: https://bpt.web.cern.ch/lhc/supertable/2023/static_lhc_supertable.html

In [ ]:
tree.GetEvent(7)
tree.EventHeader.GetEventNumber()

In [ ]:
tree.Digi_MuFilterHits.GetEntries()

In [ ]:
tree.Digi_MuFilterHits.Dump()

Get the MuFi system, plane and orientation.

system name = numbering in GetSystem()

### Veto = 1

### US = 2

### DS = 3

We will have to work a bit more to get the bar number :-D

In [ ]:
for aHit in tree.Digi_MuFilterHits: # loop over hits
    print(aHit.GetSystem(), aHit.GetDetectorID(), aHit.GetPlane(), aHit.isVertical(), aHit.GetDetectorID()%1000)

The plane numbering in the sndsw starts from 0! So does the bar count!


Lets read out the hardware information from the hits - the tofpet_id, tofpet_channel, the board_id, etc.

In [ ]:
for aHit in tree.Digi_MuFilterHits: # loop over MuFilter hits
    # loop over all channels per bar
    for side in range(aHit.GetnSides()): # loop over sides - left/right or top
        for channel in range(aHit.GetnSiPMs()): # loop over channels per side
            ch = 8*side + channel
            print(aHit.GetBoardID(ch), aHit.GetTofpetID(ch), aHit.Getchannel(ch), aHit.GetSystem(), aHit.GetDetectorID(),side, channel)
            #print(aHit.GetBoardID(ch), aHit.GetTofpetID(ch), aHit.Getchannel(ch), round(aHit.GetSignal(ch),2), aHit.GetSystem(), aHit.GetDetectorID(),aHit.isVertical(), side, channel)

The channel numbering in the sndsw starts from 0!
The default value for QDC per channel is -999! What happens is: if at least 1 SiPM fires, a hit is created with all other channels assigned QDC=-999. 
If another SiPM of the same bar fires, its QDC is recorded in place of the default -999.
In TI18 data most recorded events are passing muons and the small SiPMs are unlikely to fire.

Lets read out some SciFi hits info:

In [ ]:
for aHit in tree.Digi_ScifiHits: # loop over SciFi hits
    print(aHit.GetStation(), aHit.GetMat(), aHit.GetSiPM(), aHit.GetSiPMChan(), aHit.GetChannelID(), aHit.GetDetectorID(), round(aHit.GetSignal(),2))

 Here comes our pain: negative QDC per MIP (mostly muons in TI18 data).

### Reading the MC

 The path+filename is: /eos/experiment/sndlhc/MonteCarlo/MuonBackground/muons_down/scoring_1.8_Bfield/sndLHC.Ntuple-TGeant4-160urad_magfield_2022TCL6_muons_rock_5e7pr_digCPP.root

In [ ]:
file_mc = ROOT.TFile("/eos/experiment/sndlhc/MonteCarlo/MuonBackground/muons_down/scoring_1.8_Bfield/sndLHC.Ntuple-TGeant4-160urad_magfield_2022TCL6_muons_rock_5e7pr_digCPP.root")
tree_mc = file_mc.cbmsim
tree_mc.GetEntries()

In [ ]:
tree_mc.Print()

In [ ]:
tree_mc.GetEvent(0) # get event at index 0
tree_mc.EventHeader.GetEventNumber() # get its event number

### There is a #1 mismatch btw index and event number in the MC!

In [ ]:
tree_mc.GetEvent(90)
# make the output easier to grasp - decoration is important!
from decorators import *
tree_mc.MCTrack.Dump()

 Lets inspect the MC Points:

In [ ]:
for aPoint in tree_mc.ScifiPoint:
    aPoint.Dump() # one entry at a time so no beautifying

Lets check out the energy loss in preferred units:

In [ ]:
import shipunit as u # the units module
for aPoint in tree_mc.ScifiPoint:
    if aPoint.GetTrackID() == -2:
        print(aPoint.GetEnergyLoss()/u.keV, aPoint.GetEnergyLoss()/u.eV) # printing in [keV] and [eV] for comparison

#### Particles below a preset Ecut do not have associated MCTracks (TrackID=-2 is dummy), but these particles are still propagated and their MCPoints recorded thus used to make Digi_hits
Lets check if that is true!
To do so, we will access the list of MC points used to generate digitized hits.

In [ ]:
istrue = False
for aHit in tree_mc.Digi_ScifiHits: # loop over digi hits
    # loop over mc points used to make the digi hit using the unique ID == detID
    for mc_point_i, _ in tree_mc.Digi_ScifiHits2MCPoints[0].wList(aHit.GetDetectorID()) : 
            #ask if the track id for the point is -2   
            if tree_mc.ScifiPoint[mc_point_i].GetTrackID()==-2:
                istrue = True
                break
    if istrue:
        break
print(istrue)    

### Lets read some geofile
The geofile + path is /eos/experiment/sndlhc/convertedData/physics/2023/geofile_sndlhc_TI18_V3_2023.root

In [ ]:
import SndlhcGeo
geo = SndlhcGeo.GeoInterface("/eos/experiment/sndlhc/convertedData/physics/2023/geofile_sndlhc_TI18_V3_2023.root")
Scifi = geo.modules['Scifi']
MuFi = geo.modules['MuFilter']

In [ ]:
Scifi.Dump()

Reading the geo parameters: example reading the speed of light in the fibers

In [ ]:
Scifi.GetConfParF('Scifi/signalSpeed') # reading the detector object

In [ ]:
geo.snd_geo['Scifi']['signalSpeed'] # reading the geo as a dictionaty

One can also read as a dictionary with key-value pairs: example relative mat alignment parameter "LocM100"

In [ ]:
#geo.snd_geo['Scifi'].keys()
for key in geo.snd_geo['Scifi'].keys():
    if 'LocM100' in key:
        print(key)

Lets compare the values for two different tags:

In [ ]:
print(Scifi.GetConfParF('Scifi/LocM100t_4575'))
print(Scifi.GetConfParF('Scifi/LocM100t_6305'))

Lets get the coordinates of a hit - it means getting the coordinates of the sensitive volume that registerred the hit
For Scifi the function is called GetSiPMPosition, but for MuFilter it is GetPosition.

In [ ]:
A, B = ROOT.TVector3(), ROOT.TVector3()
for i, aHit in enumerate(tree.Digi_ScifiHits):
    Scifi.GetSiPMPosition(aHit.GetDetectorID(),A,B)
    A.Print()# left
    B.Print()# right
    if i==0: 
        break # only print the a few hits for simplicity of the output

Wait a bit... did we forget something here?

Any suggestions what else to try?